In [1]:
import ibmJupyterNotebookStyles
ibmJupyterNotebookStyles.apply_ibm_styles()

# Classical SVM

Support vector machines (SVMs) are a class of linear algorithms that can be used for supervised classification (SVC), regression (SVR), density estimation, and novelty detection. For example, we use binary classification to simplify derivations below. The algorithm finds a hyperplane that separates the data into two classes by maximizing the margin. As a result, the SVM has good generalization accuracy on unseen data and allows for specialized optimization methods that learn from a large amount of data.<sup>1</sup> 

 
We will start by formalizing the binary classification problem mathematically. This includes describing the linear model for the margin. Next, primal and dual formulation are introduced for margin optimization. Next, we show that the dual formulation, combined with feature spaces, results in the kernel trick, which gives the advantage of using higher-dimensional spaces and reducing computational resources. Finally, we show two examples of using polynomial and RBF kernels in the Python code. These two types of kernels are used to solve a multiclass classification problem. 

## Decision boundary and margin

For simplicity, let’s consider an example in two dimensions. It is straightforward to generalize to the multivariate case. The objective of a linear classifier is to separate the plane into two regions, namely, region A (blue points) and region B (black points). If the point falls in region A, it belongs to the positive class. Otherwise it belongs to the negative class. The key objective is to determine the best way to separate the plane into these two regions. 

First, a straight line is given by $(w \cdot x) + b = 0$. Here, $(w \cdot x)$ is the inner product between $w = (w_1, w_2)^T$ and $x = (x_1, x_2)^T$ that separates the plane into two regions. The two regions are $(w \cdot x) + b \leq 0$ and $(w \cdot x) + b >0$. This straight line is sometimes referred to as a hyperplane. Now, the value of any point $x$ is determined by:

$$y = f(x) = \text{sign}((w \cdot x) + b),$$

where $sign(a)$ is a sign function, which equals $1$ if $a > 0$ and $-1$ otherwise. In other words, $-1$ corresponds to a negative class and $1$ corresponds to a positive class. The following figure shows a linearly separable problem in $\mathbb{R}^2$. In this figure, the support lines ($l_2,l_3$) and support vectors are indicated. Hence, the optimal decision boundary lies in the middle of the two support lines. The distance between the support lines is called the margin. The margin is given by $2/\Vert{w}\Vert$, that is, twice the distance from the decision boundary to the support vectors.<sup>2</sup> 

<img src="figs/SVM.png" alt="drawing" width="500"/><br>

## Optimization of maximal margin

To find the best decision boundary, the margin needs to be maximized. Hence, this is the optimization problem that the SVM tries to solve. We explain two methods to solve this optimization problem: the primal formulation, which can be solved by quadratic programming, and the dual formulation, which has an advantage over primal formulation and plays a key role in the kernel trick discussed later. The primal formulation is given for completeness, but we will be building on the dual formulation.

### Primal optimization problem

Finding the maximal margin results in the following optimization problem:

$$\text{max}_{b,w} \frac{2}{\Vert{w}\Vert},$$
such that $$(w\cdot x + b) \geq 1, \forall i:y_i = 1,$$
$$(w\cdot x + b) \leq -1, \forall i:y_i = -1.$$

We can write this problem as finding the minimum of

$$\text{min}_{b,w} \frac{\Vert{w}\Vert^{2}}{2},$$
such that $$y_i((w \cdot x_i) + b)\geq 1, i = 1, \dots, n.$$

To solve this optimization problem, the Lagrange multiplier can be used as follows:

$$L(w, b, \alpha) = \frac{\Vert{w}\Vert^{2}}{2} - \sum_{i}^N \alpha_i (y_i(w^{T}x_i+b) -1),$$

where the Lagrange multiplier vector is given by $\alpha = (\alpha_1, ..., \alpha_N)^T$. To obtain the best decision boundary, the margin needs to be maximized. This results in an optimization problem that finds the optimal values for $w$ and $b$ as follows:

$$\text{min}_{\alpha, w, b} L_{p} = \frac{\Vert{w}\Vert^{2}}{2} - \sum_{i}^N \alpha_i (y_i(w^{T}x_i+b) -1).$$

This is called the primal optimization problem of the SVM. Solving the primal optimization problem requires high computational power.<sup>2</sup> The dual formulation of this optimization problem reduces the computational resources needed. 

### Dual optimization problem

Deriving the dual formulation of the primal problem is beyond the scope of this course. Readers interested in the derivation of the dual optimization problem are advised to review section 12.2.1 of Hastie, Tibshirani, and Friedman 2016.<sup>3</sup> 

The dual representation of the linear classifier is given by: 

$$f(x) = \sum_i^N\alpha_iy_i\langle x_i, x\rangle + b.$$

The optimization problem remains the same for the dual formulation as for the primal formulation, that is:

$$L_d = \text{max}_{\alpha > 0} \sum_{i} \alpha_{i}- \frac{1}{2} \sum_{j,k} \alpha_j \alpha_k y_jy_k \langle x_j, x_k\rangle.$$

Next, we will explain the kernel trick, which leverages the inner product $\langle x_j, x_k\rangle$ of the dual formulation. 



## Kernels

Kernels are built on the concepts of dual formulation of the optimization problem, as described in the previous section, and feature maps. A feature map is a function that maps data to a higher-dimensional feature space. Before discussing the kernel trick, we will further explain feature maps.  

### Feature space

A key concept in kernel methods is that of a feature space. Data cannot typically be separated by a decision boundary in its original space. A common technique used to find such a decision boundary consists of applying a nonlinear transformation function to the data. This function is called a feature map, as it transforms the raw inputs to features in the higher-dimensional feature space. 

<img src="figs/featuremaps.png" alt="drawing" width="500"/><br>

### Kernel trick

We have seen how higher-dimensional transformations allow us to separate data to make classification predictions. To train a support vector classifier and optimize our objective function, we have to perform operations with the higher-dimensional vectors in the transformed feature space. In real applications, there might be many features in the data, and applying transformations that involve many polynomial combinations of these features will lead to extremely high and impractical computational costs.

However, the dual formulation provides a way to use feature spaces without having to compute them. The dual formulation requires only the dot product of the feature spaces:

$$L_d = \text{max}_{\alpha > 0} \sum_{i} \alpha_{i}- \frac{1}{2} \sum_{j,k} \alpha_j \alpha_k y_j y_k \langle\phi(x_j), \phi(x_k)\rangle,$$

where a function that takes as its input vectors in the original space and returns the dot product of the vectors in the feature space is called a kernel function. That is, $K = \langle \phi(x_j), \phi(x_k)\rangle$. Implicitly mapping data to higher dimensions using kernels is advantageous because operational cost is lower and nonlinearly separable data in the original space can be classified. The field of kernel research is looking to feature spaces for better mapping. 

The next section will discuss several kernel functions. 


## Kernel-based methods

Several functions exist to map input data to a feature space. In this section, we discuss the radial-based function (RBF) and the polynomial function. Additionally, we show how to apply them to the Iris data set using `scikit-learn`. 

So far, we have discussed only SVM used for binary classification problems. However, SVM can be used for multiclass classification as well. There are two possible approaches: one-vs-one and one-vs-rest. The details of these approaches are beyond the scope of this module. Interested readers are referred to Deng, Tian, and Zhang 2012, page 292.<sup>2</sup>

With the Iris data set, we will try to solve a multiclass classification problem. The table below shows the first five rows of the Iris data set. This data contains four features, and each data point is assigned to a class. In the following sections, we try to classify the data to the type of iris.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Import dataset
irisdata_raw=load_iris()

# Assign colum names to the dataset
colnames = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

# Read dataset to pandas dataframe
irisdata = pd.DataFrame( \
                           np.concatenate([irisdata_raw.data, \
                                           irisdata_raw.target.reshape(len(irisdata_raw.target),1)], \
                                          axis=1),
                           columns = colnames)
irisdata["Class"] = irisdata["Class"].apply(lambda x: irisdata_raw.target_names[int(x)])
print(irisdata.head())

#pre-processing
X = irisdata.drop('Class', axis=1)
y = irisdata['Class']

# Creat test and train dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

   sepal-length  sepal-width  petal-length  petal-width   Class
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


### Polynomial kernel

A polynomial kernel is a kernel function based on polynomials of the input data, namely:

$$k(x_i,x_j)=(\textbf{x}_i\cdot \textbf{x}_j + a)^b,$$

where $a$ is a constant and $b$ is the polynomial degree. The polynomial kernel is popular in image processing.<sup>4</sup> 


In [3]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

svclassifier = SVC(kernel='poly', degree=8)
svclassifier.fit(X_train, y_train)

# Make prediction
y_pred = svclassifier.predict(X_test)

# Evaluate the Algorithm
print("Polynomial Kernel - Confusion Matrix")
print(confusion_matrix(y_test, y_pred))
print("Polynomial Kernel - Classification Report")
print(classification_report(y_test, y_pred))


Polynomial Kernel - Confusion Matrix
[[10  0  0]
 [ 0  7  2]
 [ 0  0 11]]
Polynomial Kernel - Classification Report
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.78      0.88         9
   virginica       0.85      1.00      0.92        11

    accuracy                           0.93        30
   macro avg       0.95      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



### Radial basis function kernel

The Gaussian radial basis function (RBF) kernel is the default kernel of SVM in `scikit-learn`. The kernel function is given by:

$$k(x_i,x_j) = e^{-\gamma\Vert{x_i-x_j}\Vert^2},$$

where $x_i$ and $x_j$ are data points, $\Vert{x_i-x_j}\Vert$ denotes Euclidean distance, and $\gamma$ is a parameter that controls the width of the Gaussian kernel. RBF is used when there is no prior knowledge of the data. 

In [4]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_train, y_train)

# Make prediction
y_pred = svclassifier.predict(X_test)

# Evaluate the Algorithm
print("Gaussian Kernel - Confusion Matrix")
print(confusion_matrix(y_test, y_pred))
print("Gaussian Kernel - Classification Report")
print(classification_report(y_test, y_pred))


Gaussian Kernel - Confusion Matrix
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
Gaussian Kernel - Classification Report
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



## References
1. <a id='Zhang2017'></a>Xinhua Zhang. “Support Vector Machines.” In *Encyclopedia of Machine Learning and Data Mining*, 2nd ed., edited by Claude Sammut and Geoffrey I. Webb, 1214–1220. Boston, MA: Springer, 2017. https://doi.org/10.1007/978-1-4899-7687-1_810.

2. <a id='Deng2012'></a>Naiyang Deng, Yingjie Tian, and Chunhua Zhang. *Support Vector Machines: Optimization Based Theory, Algorithms, and Extensions*. Boca Raton, FL: CRC Press, 2012.

3. <a id='Friedman2017'></a>Trevor Hastie, Robert Tibshirani, and Jerome Friedman. *The Elements of Statistical Learning: Data Mining, Inference, and Prediction*, 2nd ed. New York: Springer, 2016, https://web.stanford.edu/~hastie/Papers/ESLII.pdf.

4. <a id='Muller2016'></a>Andreas C. Müller and Sarah Guido. *Introduction to Machine Learning with Python: A Guide for Data Scientists.* Sebastopol, CA: O’Reilly Media, 2016.<br>

<span style="font-size:10pt; font-weight:bold;"> &copy; Copyright IBM Corp. 2023 <br/>This content is categorized as an “existing” IBM asset as part of the IBM Quantum Accelerator contract. </span>